Sequoia package for kinship reconstruction

In [1]:
#Make sure R can find packages installed through Conda
.libPaths('/hb/home/jbos/.conda/envs/vcfR')
.libPaths("/hb/home/jbos/.conda/envs/vcfR/lib/R/library")

In [2]:
library(sequoia)
library(vcfR)
library(sf)


   *****       ***   vcfR   ***       *****
   This is vcfR 1.15.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****


Linking to GEOS 3.12.2, GDAL 3.9.2, PROJ 9.5.0; sf_use_s2() is TRUE



In [3]:
#Function to make input for Sequoia, despite the fact that our samples don't have sex or birth year
makeLifeHistory<-function(vcf){
    lh<-as.data.frame(colnames(vcf@gt)[2:length(colnames(vcf@gt))])
    lh$Sex<-4
    lh$BirthYear<--1
    colnames(lh)<-c('ID','Sex','BirthYear')
    return(lh)
    }

In [4]:
#Calculate great circle distances from lat/lons
wide_dists <- function(sites){
    x<-st_as_sf(sites,coords=c('LONGITUDE','LATITUDE'))
    x<-st_set_crs(x,4326)
    d <- st_distance(x)
    colnames(d)<-sites$Name
    rownames(d)<-sites$Name
    d<-as.data.frame(d)
    return(d)
}

In [24]:
#Check distance between potential relatives
checkDist<-function(mayberel,dists,PairNumber){
    pn<-PairNumber
    p<-unlist(strsplit(mayberel$MaybePar$ID1[pn],"_"))[1]
    o<-unlist(strsplit(mayberel$MaybePar$ID2[pn],"_"))[1]
    return(dists[p,o])
    }

In [11]:
#Create SNPs.txt file for each taxon
sequoia1<-GenoConvert(InFile="/hb/scratch/jbos/spp1/vcf_thinned500bp.recode.vcf",OutFile='snps1.txt',OutFormat='seq')
sequoia2<-GenoConvert(InFile="/hb/scratch/jbos/spp2/vcf_thinned500bp.recode.vcf",OutFile='snps2.txt',OutFormat='seq')
sequoia3<-GenoConvert(InFile="/hb/scratch/jbos/spp3/vcf_thinned500bp.recode.vcf",OutFile='snps3.txt',OutFormat='seq')
sequoia4<-GenoConvert(InFile="/hb/scratch/jbos/spp4/vcf_thinned500bp.recode.vcf",OutFile='snps4.txt',OutFormat='seq')

ERROR: Error in GenoConvert(InFile = "/hb/scratch/jbos/spp1/vcf_thinned500bp.recode.vcf", : rows have unequal length


In [12]:
#Read in SNP data for each taxon
vcf1<-read.vcfR("/hb/scratch/jbos/spp1/vcf_thinned500bp.recode.vcf")
vcf2<-read.vcfR("/hb/scratch/jbos/spp2/vcf_thinned500bp.recode.vcf")
vcf3<-read.vcfR("/hb/scratch/jbos/spp3/vcf_thinned500bp.recode.vcf")
vcf4<-read.vcfR("/hb/scratch/jbos/spp4/vcf_thinned500bp.recode.vcf")

Scanning file to determine attributes.
File attributes:
  meta lines: 1605
  header_line: 1606
  variant count: 606
  column count: 133
Meta line 1605 read in.
All meta lines processed.
gt matrix initialized.
Character matrix gt created.
  Character matrix gt rows: 606
  Character matrix gt cols: 133
  skip: 0
  nrows: 606
  row_num: 0
Processed variant: 606
All variants processed
Scanning file to determine attributes.
File attributes:
  meta lines: 1605
  header_line: 1606
  variant count: 640
  column count: 82
Meta line 1605 read in.
All meta lines processed.
gt matrix initialized.
Character matrix gt created.
  Character matrix gt rows: 640
  Character matrix gt cols: 82
  skip: 0
  nrows: 640
  row_num: 0
Processed variant: 640
All variants processed
Scanning file to determine attributes.
File attributes:
  meta lines: 1605
  header_line: 1606
  variant count: 798
  column count: 69
Meta line 1605 read in.
All meta lines processed.
gt matrix initialized.
Character matrix gt create

In [13]:
#Make mostly empty life history inputs for Sequoia functions
lh1<-makeLifeHistory(vcf1)
lh2<-makeLifeHistory(vcf2)
lh3<-makeLifeHistory(vcf3)
lh4<-makeLifeHistory(vcf4)

In [14]:
#Make genome inputs
sequoia1<-as.matrix(read.table("snps1.txt", row.names=1, header=FALSE))
sequoia2<-as.matrix(read.table("snps2.txt", row.names=1, header=FALSE))
sequoia3<-as.matrix(read.table("snps3.txt", row.names=1, header=FALSE))
sequoia4<-as.matrix(read.table("snps4.txt", row.names=1, header=FALSE))

In [16]:
#Find potential relatives with no age prior
MaybeRel1 <- GetMaybeRel(GenoM = sequoia1, Err = 0.005, LifeHistData = lh1, quiet=FALSE,Herm='B')

Searching for non-assigned parent-offspring pairs ... (Module = par)

Warning message:
“ There are 2 monomorphic (fixed) SNPs, these will be excluded 
”
After exclusion, There are  124  individuals and  549  SNPs.


Ageprior: Flat 0/1, overlapping generations, MaxAgeParent = 99,99



Counting opposing homozygous loci between all individuals ...
Checking for non-assigned Parent-Offspring pairs ...
Checking for Parent-Parent-Offspring trios ...


Found 2 likely parent-offspring pairs, and 0 other non-assigned pairs of possible relatives



In [17]:
#Find potential relatives with no age prior
MaybeRel2 <- GetMaybeRel(GenoM = sequoia2, Err = 0.005, LifeHistData = lh2, quiet=FALSE,Herm='B')

Searching for non-assigned parent-offspring pairs ... (Module = par)

Warning message:
“ There are 1 monomorphic (fixed) SNPs, these will be excluded 
”
After exclusion, There are  73  individuals and  602  SNPs.


Ageprior: Flat 0/1, overlapping generations, MaxAgeParent = 99,99



Counting opposing homozygous loci between all individuals ...
Checking for non-assigned Parent-Offspring pairs ...


Found 1 likely parent-offspring pairs, and 0 other non-assigned pairs of possible relatives



In [18]:
#Find potential relatives with no age prior
MaybeRel3 <- GetMaybeRel(GenoM = sequoia3, Err = 0.005, LifeHistData = lh3, quiet=FALSE,Herm='B')

Searching for non-assigned parent-offspring pairs ... (Module = par)

Genotype matrix looks OK! There are  60  individuals and  750  SNPs.


Ageprior: Flat 0/1, overlapping generations, MaxAgeParent = 99,99



Counting opposing homozygous loci between all individuals ...
Checking for non-assigned Parent-Offspring pairs ...
Checking for Parent-Parent-Offspring trios ...


Found 3 likely parent-offspring pairs, and 0 other non-assigned pairs of possible relatives



In [19]:
#Find potential relatives with no age prior
MaybeRel4 <- GetMaybeRel(GenoM = sequoia4, Err = 0.005, LifeHistData = lh4, quiet=FALSE,Herm='B')

Searching for non-assigned parent-offspring pairs ... (Module = par)

Genotype matrix looks OK! There are  69  individuals and  404  SNPs.


Ageprior: Flat 0/1, overlapping generations, MaxAgeParent = 99,99



Counting opposing homozygous loci between all individuals ...
Checking for non-assigned Parent-Offspring pairs ...


Found 1 likely parent-offspring pairs, and 0 other non-assigned pairs of possible relatives



In [20]:
MaybeRel1

,ID1,ID2,TopRel,LLR,OH,BirthYear1,BirthYear2,AgeDif,Sex1,Sex2,SNPdBoth
,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,LEY11_T15,LEY04_T15,PO,5.30,0,NA,NA,NA,4,4,544
2,LEY07_T02,CEB35_T08,PO,1.78,0,NA,NA,NA,4,4,367


In [21]:
MaybeRel2

,ID1,ID2,TopRel,LLR,OH,BirthYear1,BirthYear2,AgeDif,Sex1,Sex2,SNPdBoth
,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,LEY14_T09,CEB07_T14,PO,8.62,0,NA,NA,NA,4,4,591


In [76]:
MaybeRel3

$MaybePar
        ID1       ID2 TopRel  LLR OH BirthYear1 BirthYear2 AgeDif Sex1 Sex2
1 CEB01_T12 CEB23_T16     PO 6.52  0         NA         NA     NA    4    4
2 CEB01_T11 CEB01_T12     PO 6.04  0         NA         NA     NA    4    4
3 CEB01_T11 CEB23_T16     PO 5.39  1         NA         NA     NA    4    4
  SNPdBoth
1      681
2      719
3      676

$MaybeTrio
         id   parent1   parent2 LLRparent1 LLRparent2 LLRpair OHparent1
1 CEB23_T16 CEB01_T11 CEB01_T12       -0.3      -3.89    0.28         1
  OHparent2 MEpair SNPd.id.parent1 SNPd.id.parent2
1         0      5             676             681

In [77]:
MaybeRel4

,ID1,ID2,TopRel,LLR,OH,BirthYear1,BirthYear2,AgeDif,Sex1,Sex2,SNPdBoth
,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,CEB26_T22,CEB05_T12,PO,2.97,0,NA,NA,NA,4,4,396


In [22]:
#Import site locations with lat/lons
sites <- read.csv("all_Atenuis_sites_FIXED.csv")

In [23]:
#Create distance matrix from sites
dists<-wide_dists(sites)

In [25]:
#Check distance between relatives and convert to km, taxa 1
checkDist(MaybeRel1,dists,1)/1000
checkDist(MaybeRel1,dists,2)/1000

58.33163 [m]

44.55889 [m]

In [26]:
#Check distance between relatives and convert to km, taxa 2
checkDist(MaybeRel2,dists,1)/1000

142.4513 [m]

In [27]:
#Check distance between relatives and convert to km, taxa 3
checkDist(MaybeRel3,dists,1)/1000
checkDist(MaybeRel3,dists,2)/1000
checkDist(MaybeRel3,dists,3)/1000

180.5587 [m]

0 [m]

180.5587 [m]

In [28]:
#Check distance between relatives and convert to km, taxa 4
checkDist(MaybeRel4,dists,1)/1000

174.1064 [m]